In [1]:
import numpy as np
import pandas as pd
import polars as pl
from sklearn.model_selection import train_test_split
from interpret import show
from interpret.perf import ROC

from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

# df = pd.read_csv(
#     "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
#     header=None)
# df.columns = [
#     "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
#     "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
#     "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
# ]
# X = df.iloc[:, :-1]
# y = (df.iloc[:, -1] == " >50K").astype(int)

# seed = 42
# np.random.seed(seed)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

In [59]:
from ml.data import load_penguins
df = load_penguins()
df = df.drop_nulls()

X = df.drop('island')
y = X.drop_in_place('species')

X_train, X_test, y_train, y_test = train_test_split(X.to_pandas(), y.to_pandas(), test_size=0.20, random_state=42)

In [58]:
X


bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
f64,f64,f64,f64,str
39.1,18.7,181.0,3750.0,"""male"""
39.5,17.4,186.0,3800.0,"""female"""
40.3,18.0,195.0,3250.0,"""female"""
36.7,19.3,193.0,3450.0,"""female"""
39.3,20.6,190.0,3650.0,"""male"""
…,…,…,…,…
55.8,19.8,207.0,4000.0,"""male"""
43.5,18.1,202.0,3400.0,"""female"""
49.6,18.2,193.0,3775.0,"""male"""


In [55]:
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X_train, y_train, name='Train Data')
show(hist)

In [42]:
from interpret.glassbox import ExplainableBoostingClassifier

ebm = ExplainableBoostingClassifier()
ebm.fit(X_train, y_train)

c:\Dev\interpretable-ml\.venv\Lib\site-packages\interpret\glassbox\_ebm\_ebm.py:1173: UserWarning:

Detected multiclass problem. Forcing interactions to 0. Multiclass interactions only have local explanations. They are not currently displayed in the global explanation visualizations. Set interactions=0 to disable this warning. If you still want multiclass interactions, this API accepts a list, and the measure_interactions function can be used to detect them.



ExplainableBoostingClassifier()

In [ ]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

In [ ]:
# FIXME Can't do multiclass scoring
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
ebm_perf = ROC(ebm).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

Exception: Only binary classification supported in the ROC class